Here we explain how to use matrices to convert from angle space $2\Theta$, $\theta$, $\chi$, $\phi$  to reciprocal space. The proceedure we follow is similar to that described in Busing and Levy Acta Cryst. 22, 457 (1967).  

The first thing to do is to define the unit cell and reciprocal lattice and we choose an x-ray wavelength corresponding to 10 keV. Lengths here are in Angstroms. We use the "physics" convention where $|a^*|=2\pi/a$ rather than the "crystalographic" notation of $|a^*|=2\pi/a$. Angles are in degrees. 

In [1]:
import numpy as np
from trigonometry import sin, cos, arccos

lam = 12398/10000

a = 5.811
b = 10.07
c = 6.628
alpha = 90
beta =  100.7
gamma = 90

M = np.array([[a**2,           a*b*cos(gamma), a*c*cos(beta)],
              [a*b*cos(gamma), b**2,           b*c*cos(alpha)],
              [a*c*cos(beta),  b*c*cos(alpha), c**2          ]])

Minv = (2*np.pi)**2*np.linalg.inv(M)

a_star = np.sqrt(Minv[0, 0])
b_star = np.sqrt(Minv[1, 1])
c_star = np.sqrt(Minv[2, 2])

alpha_star = arccos(Minv[1, 2]/(b_star*c_star))
beta_star = arccos(Minv[0, 2]/(a_star*c_star))
gamma_star = arccos(Minv[0, 1]/(a_star*b_star))

To proceed with our conversion it is helpful to imagine a Cartesian frame of reference as an intermediate step between angle and reciprocal space. 

To do this we introduce $\omega$ as an angle that defines the direction of $Q$ and use Busing and Levy's construction. Here the order of rotations i.e. $\phi$ sits on $\chi$  sits on $\theta$ is crucial, but this Cartesian space overall can be oriented in any direction -- this will get absorbed into our workflow. 

In [2]:
def angles2cart(tth, th, chi, phi, lam):
    omega = th - tth/2
    q_carterian_unit_vector = np.array([cos(omega)*cos(chi)*cos(phi) - sin(omega)*sin(phi),
                                        cos(omega)*cos(chi)*sin(phi) + sin(omega)*cos(phi),
                                        cos(omega)*sin(chi)])
    
    q_cart = 4*np.pi*sin(tth/2)*q_carterian_unit_vector/lam
    return q_cart

Converting from reciprocal lattice vectors to a Cartesian frame can be done by a matrix. The 2pi here in the bottomm right of the matrix comes from our convention. 

In [3]:
B = np.array([[a_star, b_star*cos(gamma_star), c_star*cos(beta_star)],
              [0,       b_star*sin(gamma_star), -c_star*sin(beta_star)*cos(alpha)],
              [0,       0,                       2*np.pi/c]])   

To uniquely determine the orientation of a sample you need to know a minimum of two non-parallel reflections definied in terms of angular positions of the goniometer and reciprocal space assignments. Both the angular position and the reciprocal space assignments are converted into Cartesian space. Some matrix tricks can be used to determine matrix U, which defnies the rotational offset between the vectors in Cartesian space. In effect, the procedure choose U such that ``position_1`` to have $Q$ exactly parallel to ``assignment_1`` and chooses the azimuthal translation frame to match ``position_1`` to ``assignment_1``. This will only work perfectly if the positioins and assignment have the same offsets. 

In [4]:
position_1 = (44.7580,  22.3790,  90.0000, 0)
position_2 = (55.8185,   14.7355, 90.0000, 0)

assignment_1 = np.array([0, 0, 4])
assignment_2 = np.array([-1, 0, 5])

def get_T_mat(vec1, vec2):
    t1 = vec1/np.linalg.norm(vec1)
    cross = np.cross(vec1, vec2)
    t3 = cross/np.linalg.norm(cross)
    t2 = np.cross(t3, t1)
    T = np.stack([t1, t2, t3]).T
    return T

def get_U(position_1, assignment_1,
          position_2, assignment_2,
          B, lam):
    T_pos = get_T_mat(angles2cart(*position_1, lam),
                      angles2cart(*position_2, lam))
    T_ass = get_T_mat(np.dot(B, assignment_1),
                      np.dot(B, assignment_2))
    U = np.matmul(T_pos, T_ass.T)
    return U

U = get_U(position_1, assignment_1,
          position_2, assignment_2,
          B, lam)
UB = np.matmul(U, B)
invUB = np.linalg.inv(UB)

def get_HKL(position, invUB, lam):
    return np.dot(invUB, angles2cart(*position, lam)) 

for p, a in zip([position_1, position_2],
                [assignment_1, assignment_2]):
                  predict = get_HKL(p, invUB, lam)
                  print(f"Assignment {a.round(4)} predicted as {predict.round(4)}")

Assignment [0 0 4] predicted as [ 0. -0.  4.]
Assignment [-1  0  5] predicted as [-1.  0.  5.]
